# Dự báo thời tiết Hà Nội

## 1. Tải các gói thư viện

In [16]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

## 2. Đọc dữ liệu

In [17]:
df = pd.read_excel("C:\doAn2\data\weatherbit_hanoi_2024.xlsx")
df.tail(5)

,Mốc thời gian,Mức độ mây che phủ,Điểm sương,Bức xạ khuếch tán ngang(DHI),Bức xạ trực tiếp bình thường(DNI),Bức xạ mặt trời toàn phần ngang(GHI),max_dhi,max_dni,max_ghi,Bức xạ mặt trời tổng,...,Thời điểm có hướng gió mạnh nhất,Hướng gió hiện tại,Tốc độ gió giật,Tốc độ gió trung bình,Lượng mưa ghi nhận,Lượng mưa từ nguồn vệ tinh GPM,Áp suất không khí,Độ ẩm tương đối,Áp suất mặt biển chuẩn,Trạng thái cập nhật dữ liệu
360,2024-12-26,92,14.3,40,327,206,116,910,759,111,...,1735185600,233,6.4,1.4,0.0,0.0,1021,66,1021,final
361,2024-12-27,100,13.6,40,328,206,116,910,760,52,...,1735254000,212,7.6,2.2,0.0,0.0,1023,73,1024,final
362,2024-12-28,65,11.0,40,328,206,116,910,761,174,...,1735365600,230,8.0,2.2,0.0,0.0,1025,65,1026,final
363,2024-12-29,39,11.8,40,328,207,116,911,762,207,...,1735452000,136,6.8,1.7,0.0,0.0,1020,68,1021,final
364,2024-12-30,85,13.8,40,328,207,116,911,763,55,...,1735538400,295,5.6,1.0,0.0,0.0,1019,78,1020,final


## 3. Tiền xử lý dữ liệu

In [18]:
df.shape

(365, 34)

In [19]:
pd.set_option('display.max_rows', 500)
df.dtypes

Mốc thời gian                                           datetime64[ns]
Mức độ mây che phủ                                               int64
Điểm sương                                                     float64
Bức xạ khuếch tán ngang(DHI)                                     int64
Bức xạ trực tiếp bình thường(DNI)                                int64
Bức xạ mặt trời toàn phần ngang(GHI)                             int64
max_dhi                                                          int64
max_dni                                                          int64
max_ghi                                                          int64
Bức xạ mặt trời tổng                                             int64
Giá trị dự đoán bức xạ khuếch tán ngang(DHI)                     int64
Giá trị dự đoán bức xạ trực tiếp bình thường(DNI)                int64
Giá trị dự đoán bức xạ mặt trời toàn phần ngang(GHI)             int64
Giá trị dự đoán bức xạ mặt trời tổng                             int64
Nhiệt 

In [20]:
pd.set_option('display.precision', 3)
df.describe()

,Mốc thời gian,Mức độ mây che phủ,Điểm sương,Bức xạ khuếch tán ngang(DHI),Bức xạ trực tiếp bình thường(DNI),Bức xạ mặt trời toàn phần ngang(GHI),max_dhi,max_dni,max_ghi,Bức xạ mặt trời tổng,...,Hướng gió tại thời điểm gió mạnh nhất,Thời điểm có hướng gió mạnh nhất,Hướng gió hiện tại,Tốc độ gió giật,Tốc độ gió trung bình,Lượng mưa ghi nhận,Lượng mưa từ nguồn vệ tinh GPM,Áp suất không khí,Độ ẩm tương đối,Áp suất mặt biển chuẩn
count,365,365.000,365.000,365.000,365.000,365.000,365.000,365.000,365.000,365.000,...,365.000,3.650e+02,365.000,365.000,365.000,365.000,365.000,365.000,365.000,365.000
mean,2024-06-30 22:05:35.342465792,70.285,21.291,47.485,378.540,289.800,123.542,935.022,955.005,146.030,...,180.342,1.720e+09,180.342,7.274,1.576,6.733,6.733,1009.608,79.364,1010.510
min,2023-12-31 00:00:00,0.000,3.300,40.000,326.000,205.000,115.000,909.000,757.000,21.000,...,50.000,1.704e+09,50.000,2.800,0.600,0.000,0.000,992.000,52.000,993.000
25%,2024-04-01 00:00:00,56.000,18.200,43.000,348.000,241.000,120.000,928.000,854.000,75.000,...,140.000,1.712e+09,140.000,6.000,1.200,0.000,0.000,1004.000,75.000,1004.000
50%,2024-07-01 00:00:00,75.000,23.300,48.000,382.000,304.000,126.000,937.000,1007.000,120.000,...,161.000,1.720e+09,161.000,7.200,1.500,0.500,0.500,1009.000,81.000,1010.000
75%,2024-09-30 00:00:00,90.000,25.800,52.000,412.000,339.000,126.000,942.000,1050.000,211.000,...,216.000,1.728e+09,216.000,8.400,1.800,6.500,6.500,1015.000,85.000,1016.000
max,2024-12-30 00:00:00,100.000,28.300,53.000,419.000,346.000,128.000,956.000,1062.000,337.000,...,348.000,1.736e+09,348.000,25.600,5.600,114.500,114.500,1028.000,95.000,1029.000
std,NaN,24.795,5.539,4.779,32.746,50.541,4.084,13.227,109.124,84.635,...,62.290,9.126e+06,62.290,2.019,0.511,14.651,14.651,7.100,8.757,7.124


In [21]:
print('Null Values =', df.isnull().values.any())

Null Values = False


## 4. Bài toán con: Dự báo thời tiết 1 ngày tiếp theo ở Hà Nội

### 4.0. Chuẩn bị dữ liệu cho mô hình HQTT

In [22]:
# trích xuất biến độc lập
features = [
    "Điểm sương",
    "Áp suất không khí",
    "Áp suất mặt biển chuẩn",
    "Nhiệt độ trung bình",
    "Bức xạ mặt trời toàn phần ngang(GHI)",
    "Bức xạ mặt trời tổng",
    "Lượng mưa từ nguồn vệ tinh GPM",
    "Bức xạ khuếch tán ngang(DHI)",
    "Bức xạ trực tiếp bình thường(DNI)",
    "Mức độ mây che phủ",
    "Hướng gió tại thời điểm gió mạnh nhất",
    "Tốc độ gió trung bình",
    "Tốc độ gió giật",
    "Độ ẩm tương đối",
    "Cường độ tia UV max"
]

# xác định biến phụ thuộc
targets = [
    "Nhiệt độ hiện tại", 
    "Lượng mưa ghi nhận", 
    "Độ ẩm tương đối", 
    "Hướng gió hiện tại", 
    "Tốc độ gió giật", 
    "Cường độ tia UV max"
]


### 4.1. Xây dựng mô hình Hồi Quy Tuyến Tính - Linear Regression

In [23]:
# tạo tập dữ liệu dịch chuyển để dự đoán ngày tiếp theo
df_features = df[features]
df_targets = df[targets].shift(-1)
df_model = pd.concat([df_features, df_targets], axis=1).dropna()

X = df_model[features]
y = df_model[targets]

# chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# chia tập train/test 80/20
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# huấn luyện mô hình
model = LinearRegression()
model.fit(X_train, y_train)

# dự đoán trên tập test
y_pred = model.predict(X_test)

# lấy ngày dự báo
pred_date = pd.to_datetime(df["Mốc thời gian"].max(), dayfirst=True, errors='coerce') + pd.Timedelta(days=1)
date_str = pred_date.strftime("%d/%m/%Y")

# danh sách tên cột
cols = ["Dự báo thời tiết ngày", "Nhiệt độ (°C)", "Lượng mưa (mm)", "Độ ẩm (%)", 
        "Hướng gió (°)", "Tốc độ gió (m/s)", "Cường độ UV (µW/cm²)"]

# tạo dòng dữ liệu để chèn ngày
data = [[date_str] + [f"{abs(v):.1f}" for v in y_pred[0][:6]]]

# tạo bảng kết quả và hiển thị
df_pred = pd.DataFrame(data, columns=cols)

### 4.2. Hiển thị kết quả

In [24]:
display(df_pred.style.hide(axis="index").set_properties(**{"text-align": "center"}))

Dự báo thời tiết ngày,Nhiệt độ (°C),Lượng mưa (mm),Độ ẩm (%),Hướng gió (°),Tốc độ gió (m/s),Cường độ UV (µW/cm²)
31/12/2024,28.5,4.1,79.0,83.0,185.0,6.0


## 5. Bài toán cha: Dự báo thời tiết 7 ngày tiếp theo ở Hà Nội

### 5.0. Chuẩn bị dữ liệu cho các mô hình

In [25]:
# trích xuất biến đặc trưng
features = [
    "Điểm sương", 
    "Áp suất không khí", 
    "Áp suất mặt biển chuẩn", 
    "Nhiệt độ trung bình",
    "Bức xạ mặt trời toàn phần ngang(GHI)", 
    "Bức xạ mặt trời tổng", 
    "Lượng mưa từ nguồn vệ tinh GPM",
    "Bức xạ khuếch tán ngang(DHI)", 
    "Bức xạ trực tiếp bình thường(DNI)", 
    "Mức độ mây che phủ",
    "Hướng gió tại thời điểm gió mạnh nhất", 
    "Tốc độ gió trung bình", 
    "Tốc độ gió giật",
    "Độ ẩm tương đối", 
    "Cường độ tia UV max"
]

# trích xuất biến mục tiêu
targets = [
    "Nhiệt độ hiện tại", 
    "Lượng mưa ghi nhận", 
    "Độ ẩm tương đối",
    "Hướng gió hiện tại", 
    "Tốc độ gió giật", 
    "Cường độ tia UV max"
]

# tiền xử lý
X = df[features].copy()
y = df[targets].copy()
feature_columns = X.columns.tolist()

# chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 5.1. Mô hình Multivariate Multiple Regression

In [26]:
# huấn luyện mô hình riêng cho từng biến mục tiêu
model_dict_MLR = {}
test_data_dict_MLR = {}
for col in targets:
    y_target = y[col]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_target, test_size=0.2, shuffle=False)
    model = LinearRegression()
    model.fit(X_train, y_train)
    model_dict_MLR[col] = model
    test_data_dict_MLR[col] = (X_test, y_test)

# dự báo cho 7 ngày tiếp theo
n_days = 7
ensemble_input = pd.DataFrame([(df[features].mean() + df[features].median()) / 2])
latest_input = ensemble_input.copy()

forecast_MLR = []

for day in range(n_days):
    X_input_scaled = scaler.transform(latest_input)

    raw_pred = []
    for col in targets:
        model = model_dict_MLR[col]
        value = model.predict(X_input_scaled)[0]
        raw_pred.append(value)

    # Giới hạn logic
    pred = [
        np.clip(raw_pred[0], -10, 45),    # Nhiệt độ
        np.clip(raw_pred[1], 0, 500),     # Lượng mưa
        np.clip(raw_pred[2], 0, 100),     # Độ ẩm
        np.clip(raw_pred[3], 0, 360),     # Hướng gió
        np.clip(raw_pred[4], 0, 100),     # Tốc độ gió
        np.clip(raw_pred[5], 0, 15),      # Tia UV
    ]
    forecast_MLR.append(pred)

    # cập nhật đầu vào
    colmap = {
        "Nhiệt độ trung bình": pred[0],
        "Lượng mưa từ nguồn vệ tinh GPM": pred[1],
        "Độ ẩm tương đối": pred[2],
        "Hướng gió tại thời điểm gió mạnh nhất": pred[3],
        "Tốc độ gió giật": pred[4],
        "Cường độ tia UV max": pred[5],
    }
    for key, val in colmap.items():
        latest_input.iloc[0, latest_input.columns.get_loc(key)] = val

# hiển thị kết quả
cols = ["Nhiệt độ (°C)", "Lượng mưa (mm)", "Độ ẩm (%)", 
        "Hướng gió (°)", "Tốc độ gió (m/s)", "Cường độ tia UV (µW/cm²)"]
dates = pd.date_range(df["Mốc thời gian"].max() + pd.Timedelta(days=1), periods=n_days).strftime("%d/%m/%Y")

df_out = pd.DataFrame(forecast_MLR, columns=cols)
df_out.insert(0, "Dự báo thời tiết ngày", dates)

fmt = lambda x: f"{x:.1f}"
for col in cols:
    df_out[col] = df_out[col].apply(fmt)

display(df_out.style.hide(axis="index").set_properties(**{"text-align": "center"}))

Dự báo thời tiết ngày,Nhiệt độ (°C),Lượng mưa (mm),Độ ẩm (%),Hướng gió (°),Tốc độ gió (m/s),Cường độ tia UV (µW/cm²)
31/12/2024,26.2,3.6,80.2,170.7,7.2,4.4
01/01/2025,26.2,3.6,80.2,170.7,7.2,4.4
02/01/2025,26.1,3.6,80.2,170.7,7.2,4.4
03/01/2025,26.1,3.6,80.2,170.7,7.2,4.4
04/01/2025,26.1,3.6,80.2,170.7,7.2,4.4
05/01/2025,26.1,3.6,80.2,170.7,7.2,4.4
06/01/2025,26.1,3.6,80.2,170.7,7.2,4.4


### 5.2. Mô hình Random Forest Regressor

In [27]:
# trích xuất biến đặc trưng
features = [
    "Điểm sương", 
    "Áp suất không khí", 
    "Áp suất mặt biển chuẩn", 
    "Nhiệt độ trung bình",
    "Bức xạ mặt trời toàn phần ngang(GHI)", 
    "Bức xạ mặt trời tổng", 
    "Lượng mưa từ nguồn vệ tinh GPM",
    "Bức xạ khuếch tán ngang(DHI)", 
    "Bức xạ trực tiếp bình thường(DNI)", 
    "Mức độ mây che phủ",
    "Hướng gió tại thời điểm gió mạnh nhất", 
    "Tốc độ gió trung bình", 
    "Tốc độ gió giật",
    "Độ ẩm tương đối", 
    "Cường độ tia UV max"
]

# trích xuất biến mục tiêu
targets = [
    "Nhiệt độ hiện tại", 
    "Lượng mưa ghi nhận", 
    "Độ ẩm tương đối",
    "Hướng gió hiện tại", 
    "Tốc độ gió giật", 
    "Cường độ tia UV max"
]

# tiền xử lý
X = df[features].copy()
y = df[targets].copy()

# chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# huấn luyện mô hình Random Forest Regressor
model_dict_RF = {}
test_data_dict_RF = {}
for col in targets:
    y_target = y[col]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_target, test_size=0.2, shuffle=False)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    model_dict_RF[col] = model
    test_data_dict_RF[col] = (X_test, y_test)

# dự báo 7 ngày tiếp theo
n_days = 7
ensemble_input = pd.DataFrame([(df[features].mean() + df[features].median()) / 2])
latest_input = ensemble_input.copy()

forecast_RF = []

for day in range(n_days):
    X_input_scaled = scaler.transform(latest_input)

    raw_pred = []
    for col in targets:
        model = model_dict_RF[col]
        value = model.predict(X_input_scaled)[0]
        raw_pred.append(value)

    # giới hạn logic
    pred = [
        np.clip(raw_pred[0], -10, 45),    # Nhiệt độ
        np.clip(raw_pred[1], 0, 500),     # Lượng mưa
        np.clip(raw_pred[2], 0, 100),     # Độ ẩm
        np.clip(raw_pred[3], 0, 360),     # Hướng gió
        np.clip(raw_pred[4], 0, 100),     # Tốc độ gió
        np.clip(raw_pred[5], 0, 15),      # Cường độ tia UV
    ]
    forecast_RF.append(pred)

    # cập nhật đầu vào
    colmap = {
        "Nhiệt độ trung bình": pred[0],
        "Lượng mưa từ nguồn vệ tinh GPM": pred[1],
        "Độ ẩm tương đối": pred[2],
        "Hướng gió tại thời điểm gió mạnh nhất": pred[3],
        "Tốc độ gió giật": pred[4],
        "Cường độ tia UV max": pred[5],
    }
    for key, val in colmap.items():
        latest_input.iloc[0, latest_input.columns.get_loc(key)] = val

# hiển thị kết quả
cols = ["Nhiệt độ (°C)", "Lượng mưa (mm)", "Độ ẩm (%)", 
        "Hướng gió (°)", "Tốc độ gió (m/s)", "Cường độ tia UV (µW/cm²)"]
dates = pd.date_range(df["Mốc thời gian"].max() + pd.Timedelta(days=1), periods=n_days).strftime("%d/%m/%Y")

df_out = pd.DataFrame(forecast_RF, columns=cols)
df_out.insert(0, "Dự báo thời tiết ngày", dates)

fmt = lambda x: f"{x:.1f}"
for col in cols:
    df_out[col] = df_out[col].apply(fmt)

display(df_out.style.hide(axis="index").set_properties(**{"text-align": "center"}))

Dự báo thời tiết ngày,Nhiệt độ (°C),Lượng mưa (mm),Độ ẩm (%),Hướng gió (°),Tốc độ gió (m/s),Cường độ tia UV (µW/cm²)
31/12/2024,26.2,3.7,80.0,169.9,7.2,4.4
01/01/2025,25.8,3.7,80.0,169.8,7.2,4.4
02/01/2025,25.6,3.7,80.0,169.8,7.2,4.4
03/01/2025,25.4,3.7,80.0,169.8,7.2,4.4
04/01/2025,25.4,3.7,80.0,169.8,7.2,4.4
05/01/2025,25.4,3.7,80.0,169.8,7.2,4.4
06/01/2025,25.4,3.7,80.0,169.8,7.2,4.4


### 5.3 Mô hình XGBoost

In [28]:
# huấn luyện XGBoost cho từng biến mục tiêu
model_dict_XGB = {}
test_data_dict_XGB = {}
for col in targets:
    y_target = y[col]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_target, test_size=0.2, shuffle=False)
    model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0
    )
    model.fit(X_train, y_train)
    model_dict_XGB[col] = model
    test_data_dict_XGB[col] = (X_test, y_test)


# dự báo 7 ngày tới
n_days = 7
ensemble_input = pd.DataFrame([(df[features].mean() + df[features].median()) / 2])
latest_input = ensemble_input.copy()

forecast_XGB = []

for day in range(n_days):
    X_input_scaled = scaler.transform(latest_input)

    raw_pred = []
    for col in targets:
        model = model_dict_XGB[col]
        value = model.predict(X_input_scaled)[0]
        raw_pred.append(value)

    # giới hạn logic
    pred = [
        np.clip(raw_pred[0], -10, 45),    # Nhiệt độ
        np.clip(raw_pred[1], 0, 500),     # Lượng mưa
        np.clip(raw_pred[2], 0, 100),     # Độ ẩm
        np.clip(raw_pred[3], 0, 360),     # Hướng gió
        np.clip(raw_pred[4], 0, 100),     # Tốc độ gió
        np.clip(raw_pred[5], 0, 15),      # Cường độ tia UV
    ]
    forecast_XGB.append(pred)

    # cập nhật đầu vào
    colmap = {
        "Nhiệt độ trung bình": pred[0],
        "Lượng mưa từ nguồn vệ tinh GPM": pred[1],
        "Độ ẩm tương đối": pred[2],
        "Hướng gió tại thời điểm gió mạnh nhất": pred[3],
        "Tốc độ gió giật": pred[4],
        "Cường độ tia UV max": pred[5],
    }
    for key, val in colmap.items():
        latest_input.iloc[0, latest_input.columns.get_loc(key)] = val

# hiển thị kết quả
cols = ["Nhiệt độ (°C)", "Lượng mưa (mm)", "Độ ẩm (%)", 
        "Hướng gió (°)", "Tốc độ gió (m/s)", "Cường độ tia UV (µW/cm²)"]
dates = pd.date_range(df["Mốc thời gian"].max() + pd.Timedelta(days=1), periods=n_days).strftime("%d/%m/%Y")

df_out = pd.DataFrame(forecast_XGB, columns=cols)
df_out.insert(0, "Dự báo thời tiết ngày", dates)

fmt = lambda x: f"{x:.1f}"
for col in cols:
    df_out[col] = df_out[col].apply(fmt)

display(df_out.style.hide(axis="index").set_properties(**{"text-align": "center"}))

Dự báo thời tiết ngày,Nhiệt độ (°C),Lượng mưa (mm),Độ ẩm (%),Hướng gió (°),Tốc độ gió (m/s),Cường độ tia UV (µW/cm²)
31/12/2024,26.4,3.1,80.4,176.5,7.1,4.4
01/01/2025,26.3,2.9,80.4,181.9,7.0,4.4
02/01/2025,26.3,2.9,80.4,186.1,6.8,4.4
03/01/2025,26.3,2.9,80.4,193.1,6.3,4.4
04/01/2025,26.3,2.9,80.4,196.0,6.0,4.4
05/01/2025,26.3,2.9,80.4,196.7,5.6,4.4
06/01/2025,26.3,2.9,80.4,197.7,5.6,4.4


## 6. Đánh giá đồng thời cả 3 mô hình

In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# hàm đánh giá mô hình
def evaluate_models_full(model_dicts, test_data_dicts, model_names):
    all_results = []

    for model_dict, test_dict, name in zip(model_dicts, test_data_dicts, model_names):
        for col in model_dict.keys():
            model = model_dict[col]
            X_test, y_test = test_dict[col]

            y_pred = model.predict(X_test)

            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)

            all_results.append({
                "Biến mục tiêu": col,
                "Mô hình": name,
                "MAE": mae,
                "RMSE": rmse,
                "R² Score": r2
            })

    return pd.DataFrame(all_results)

In [30]:
# đánh giá
model_dicts = [model_dict_MLR, model_dict_RF, model_dict_XGB]
test_data_dicts = [test_data_dict_MLR, test_data_dict_RF, test_data_dict_XGB]
model_names = ["Multiple Linear Regression", "Random Forest", "XGBoost"]

df_scores = evaluate_models_full(model_dicts, test_data_dicts, model_names)


# làm tròn kết quả
df_scores_formatted = df_scores.copy()
for col in ["MAE", "RMSE", "R² Score"]:
    df_scores_formatted[col] = df_scores_formatted[col].apply(lambda x: f"{x:.3f}")

# hiển thị kết bảng so sánh
display(
    df_scores_formatted.style.hide(axis=0)
    .set_properties(**{"text-align": "right"})
    .set_properties(subset=["Biến mục tiêu"], **{"text-align": "left"})
    .set_table_styles([{"selector": "th", "props": [("text-align", "right")]},
                       {"selector": "th.col0", "props": [("text-align", "left")]},
                       {"selector": "th.col1", "props": [("text-align", "center")]}])
)

Biến mục tiêu,Mô hình,MAE,RMSE,R² Score
Nhiệt độ hiện tại,Multiple Linear Regression,0.171,0.219,0.996
Lượng mưa ghi nhận,Multiple Linear Regression,0.000,0.000,1.000
Độ ẩm tương đối,Multiple Linear Regression,0.000,0.000,1.000
Hướng gió hiện tại,Multiple Linear Regression,0.000,0.000,1.000
Tốc độ gió giật,Multiple Linear Regression,0.000,0.000,1.000
Cường độ tia UV max,Multiple Linear Regression,0.000,0.000,1.000
Nhiệt độ hiện tại,Random Forest,0.327,0.431,0.986
Lượng mưa ghi nhận,Random Forest,0.058,0.266,0.998
Độ ẩm tương đối,Random Forest,0.586,1.286,0.974
Hướng gió hiện tại,Random Forest,1.445,2.593,0.999
